# Comprender e implementar redes neuronales desde cero

### Contenido


1. ¿Qué son las Redes Neuronales?

2. Implementar una Red Neuronal - Clasificación binaria

3. Implementar una Red Neuronal - Clasificación Multiclase

4. ¿Qué son las redes neuronales profundas?

5. Implementación de Redes Neuronales Convolucionales

----

1. ¿Qué son las Redes Neuronales?

Las redes neuronales son un tipo de modelos de aprendizaje automático que están diseñados para **operar de manera similar a las neuronas biológicas y al sistema nervioso humano**. 

Estos modelos se utilizan para reconocer patrones y relaciones complejas que existen dentro de un conjunto de datos etiquetado. Tienen las siguientes propiedades:

    La arquitectura central de un modelo de red neuronal se compone de una gran cantidad de nodos de procesamiento simples llamados neuronas que están interconectados y organizados en diferentes capas.

    Un nodo individual en una capa está conectado a varios otros nodos en la capa anterior y siguiente. Las entradas de una capa se reciben y procesan para generar la salida que se pasa a la siguiente capa.

    La primera capa de esta arquitectura a menudo se denomina capa de entrada que acepta las entradas, la última capa se denomina capa de salida que produce la salida y todas las demás capas entre la capa de entrada y la capa de salida se denominan capas ocultas.



### Conceptos clave en una Red Neuronal

**A. Neurona:**

Una neurona es una unidad de procesamiento única de una red neuronal que está conectada a otras neuronas diferentes en la red. Estas conexiones representan entradas y salidas de una neurona. A cada una de sus conexiones, la neurona asigna un *peso* (W) que significa la importancia de la entrada y agrega un término de sesgo (b).


**B. Funciones de activación**

Las funciones de activación se utilizan para aplicar una transformación no lineal en la entrada para asignarla a la salida. El objetivo de las funciones de activación es predecir la clase correcta de la variable de destino en función de la combinación de entrada de variables. Algunas de las funciones de activación populares son Relu, Sigmoid y TanH.

**C. Propagación hacia adelante**

El modelo de red neuronal pasa por el proceso llamado propagación directa en el que pasa las salidas de activación calculadas en la dirección adelante.

$Z = W*X + b$


$A = g(Z)$

    g es la función de activación
    A es la activación usando la entrada
    W es el peso asociado con la entrada
    B es el sesgo asociado con el nodo

**D. Cálculo de errores:**

La red neuronal aprende mejorando los valores de pesos y sesgos. El modelo calcula el error en la salida predicha en la capa final que luego se usa para hacer pequeños ajustes en los pesos y el sesgo. Los ajustes se realizan de manera que se minimice el error total. La **función de pérdida** mide el error en la capa final y la **función de costo** mide el error total de la red.

$Pérdida = Valor Real - Valor Predicho$

$Costo = Suma (Pérdida)$

**E. Propagación hacia atrás:**

El modelo de red neuronal se somete al proceso llamado **retropropagación** en el que el error se pasa a las capas hacia atrás para que esas capas también puedan mejorar los valores asociados de pesos y sesgos. Utiliza el algoritmo llamado **Gradient Descent** en el que se minimiza el error y se obtienen valores óptimos de pesos y sesgos. Este ajuste de pesos y sesgos se realiza calculando la derivada del error, la derivada de los pesos, el sesgo y restándolos de los valores originales.

---
---
2. Implementar una Red Neuronal - Clasificación Binaria

Implementemos una red neuronal básica en python para la clasificación binaria que se usa para clasificar si una imagen dada es 0 o 1.

In [1]:
#!pip install tensorflow==2.2.0

In [2]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
import pandas as pd 
import numpy as np 
import keras

2.1 Preparación del conjunto de datos

El primer paso es cargar y preparar el conjunto de datos.

In [6]:
train = pd.read_csv("../datasets/train.csv")
test = pd.read_csv("../datasets/test.csv")

# include only the rows having label = 0 or 1 (binary classification)
X = train[train['label'].isin([0, 1])]

# target variable
Y = train[train['label'].isin([0, 1])]['label']

# remove the label from X
X = X.drop(['label'], axis = 1)

2.2 Implementando una Función de Activación

Usaremos la función de activación sigmoide porque genera valores entre 0 y 1, por lo que es una buena opción para un problema de clasificación binaria.

In [7]:
# implementing a sigmoid activation function
def sigmoid(z):
    s = 1.0/ (1 + np.exp(-z))    
    return s

2.3 Definir la arquitectura de la red neuronal

Crea un modelo con tres capas: Entrada, Oculta, Salida.

In [8]:
def network_architecture(X, Y):
    # nodes in input layer
    n_x = X.shape[0] 
    # nodes in hidden layer
    n_h = 10          
    # nodes in output layer
    n_y = Y.shape[0] 
    return (n_x, n_h, n_y)

2.4 Definir los parámetros de la red neuronal

Los parámetros de la red neuronal son pesos y sesgos que necesitamos inicializar con valores cero. La primera capa solo contiene entradas, por lo que no hay pesos ni sesgos, pero la capa oculta y la capa de salida tienen un término de peso y sesgo. (W1, b1 y W2, b2)

In [9]:
def define_network_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h,n_x) * 0.01 # random initialization
    b1 = np.zeros((n_h, 1)) # zero initialization
    W2 = np.random.randn(n_y,n_h) * 0.01 
    b2 = np.zeros((n_y, 1)) 
    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}    

2.5 Implementar la propagación hacia adelante

La capa oculta y la capa de salida calcularán las activaciones utilizando la función de activación sigmoide y la pasarán en la dirección de avance. Mientras se calcula esta activación, la entrada se multiplica por peso y se suma con sesgo antes de pasarla a la función.

In [10]:
def forward_propagation(X, params):
    Z1 = np.dot(params['W1'], X)+params['b1']
    A1 = sigmoid(Z1)

    Z2 = np.dot(params['W2'], A1)+params['b2']
    A2 = sigmoid(Z2)
    return {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}    

2.6 Calcular el error de red

Para calcular el costo, un enfoque sencillo es calcular el error absoluto entre la predicción y el valor real. Pero una mejor función de pérdida es la función log que se define como:

In [11]:
def compute_error(Predicted, Actual):
    logprobs = np.multiply(np.log(Predicted), Actual)+ np.multiply(np.log(1-Predicted), 1-Actual)
    cost = -np.sum(logprobs) / Actual.shape[1] 
    return np.squeeze(cost)

2.7 Implementar la propagación hacia atrás

En la función de propagación hacia atrás, el error se pasa hacia atrás a las capas anteriores y se calculan las derivadas de los pesos y el sesgo. Luego, los pesos y el sesgo se actualizan utilizando las derivadas.

In [12]:
def backward_propagation(params, activations, X, Y):
    m = X.shape[1]
    
    # output layer
    dZ2 = activations['A2'] - Y # compute the error derivative 
    dW2 = np.dot(dZ2, activations['A1'].T) / m # compute the weight derivative 
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m # compute the bias derivative
    
    # hidden layer
    dZ1 = np.dot(params['W2'].T, dZ2)*(1-np.power(activations['A1'], 2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1, axis=1,keepdims=True)/m
    
    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

def update_parameters(params, derivatives, alpha = 1.2):
    # alpha is the model's learning rate 
    
    params['W1'] = params['W1'] - alpha * derivatives['dW1']
    params['b1'] = params['b1'] - alpha * derivatives['db1']
    params['W2'] = params['W2'] - alpha * derivatives['dW2']
    params['b2'] = params['b2'] - alpha * derivatives['db2']
    return params

2.8 Compilar y entrenar el modelo

Crea una función que compile todas las funciones clave y crea un modelo de red neuronal.

In [13]:
def neural_network(X, Y, n_h, num_iterations=100):
    n_x = network_architecture(X, Y)[0]
    n_y = network_architecture(X, Y)[2]
    
    params = define_network_parameters(n_x, n_h, n_y)
    for i in range(0, num_iterations):
        results = forward_propagation(X, params)
        error = compute_error(results['A2'], Y)
        derivatives = backward_propagation(params, results, X, Y) 
        params = update_parameters(params, derivatives)    
    return params

In [18]:
y = Y.values.reshape(1, Y.size)
x = X.T
model = neural_network(x, y, n_h = 10, num_iterations = 10)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


2.9 Predicciones

In [19]:
def predict(parameters, X):
    results = forward_propagation(X, parameters)
    print (results['A2'][0])
    predictions = np.around(results['A2'])    
    return predictions

predictions = predict(model, x)
print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

[0.80148908 0.06887999 0.90268893 ... 0.80148908 0.06887999 0.90268893]
Accuracy: 95%


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


---
---

3. Implementar una Red Neuronal - Clasificación Multiclase

En el paso anterior, implementamos una NN para la clasificación binaria en python desde cero. Las bibliotecas de Python, como sklearn, proporcionan una excelente implementación de redes neuronales eficientes que se pueden usar para implementar directamente redes neuronales en un conjunto de datos. En esta sección, implementemos una red neuronal multiclase para clasificar el dígito que se muestra en una imagen del 0 al 9

3.1 Preparación del conjunto de datos

Cortar el conjunto de datos del entrenamiento en un conjunto de entrenamiento y validación

In [20]:
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn import  metrics

Y = train['label'][:10000] # use more number of rows for more training 
X = train.drop(['label'], axis = 1)[:10000] # use more number of rows for more training 
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=42)

3.2 Entrenar el modelo

Entrene un modelo de red neuronal con 10 capas ocultas.

In [21]:
model = neural_network.MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5,), solver='lbfgs', random_state=18)
model.fit(x_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5,), random_state=18,
              solver='lbfgs')

3.3 Predicciones

In [22]:
predicted = model.predict(x_val)
print("Classification Report:\n %s:" % (metrics.classification_report(y_val, predicted)))

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       186
           1       0.96      0.86      0.91       210
           2       0.12      0.99      0.21       220
           3       0.00      0.00      0.00       190
           4       0.00      0.00      0.00       188
           5       0.00      0.00      0.00       194
           6       0.00      0.00      0.00       190
           7       0.00      0.00      0.00       233
           8       0.00      0.00      0.00       197
           9       0.00      0.00      0.00       192

    accuracy                           0.20      2000
   macro avg       0.11      0.18      0.11      2000
weighted avg       0.11      0.20      0.12      2000
:


C:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


4. Redes neuronales profundas - Redes neuronales convolucionales

Las Redes Neuronales Profundas están compuestas por muchas y complejas capas ocultas que intentan extraer características de bajo nivel de las imágenes. Algunos ejemplos de redes neuronales profundas complejas son las redes neuronales convolucionales y las redes neuronales recurrentes.


**Redes neuronales convolucionales**

En las redes neuronales convolucionales, cada entrada de imagen se trata como una matriz de valores de píxeles que representa la cantidad de oscuridad en un píxel determinado de la imagen. A diferencia de las redes neuronales tradicionales que tratan una imagen como una red unidimensional, las CNN consideran la ubicación de los píxeles y los vecinos para la clasificación.

![img](https://www.mdpi.com/information/information-07-00061/article_deploy/html/images/information-07-00061-g001.png)

### Componentes clave de la red neuronal convolucional.

**A. Capa convolucional:** en esta capa, se utiliza una matriz de kernel (o peso) para extraer características de bajo nivel de las imágenes. El núcleo con sus pesos gira sobre la matriz de la imagen en forma de ventana deslizante para obtener la salida convolucionada. La matriz del kernel se comporta como un filtro en una imagen extrayendo información particular de la matriz de la imagen original. Durante el proceso de colvolución, los pesos se aprenden de tal manera que se minimiza la función de pérdida.

**B. Stride:** Stride se define como el número de pasos que toma el kernel o la matriz de peso mientras se mueve por toda la imagen moviendo N píxeles a la vez. Si la matriz de peso mueve N píxeles a la vez, se llama Stride de N.

Créditos de imagen - www.deeplearning.net

**C. Capa de agrupación (Pooling):** las capas de agrupación (Pooling) se utilizan para extraer las características más informativas de la salida convolucionada generada.

![img](https://upload.wikimedia.org/wikipedia/commons/e/e9/Max_pooling.png)

**D. Capa de salida:** para generar la salida final, se aplica una capa densa o completamente conectada con la función de activación softmax. La función Softmax se usa para generar las probabilidades para cada clase de la variable objetivo.

5. Implementar una red neuronal de convolución

5.1 Preparación del conjunto de datos

En el primer paso, preparemos el conjunto de datos y dividámoslo en conjuntos de entrenamiento y validación. Para fines de modelado y entrenamiento, podemos usar la biblioteca de python: Keras.

In [61]:
Y = train['label']
X = train.drop(['label'], axis=1)

x_train, x_val, y_train, y_val = train_test_split(X.values, Y, test_size=0.10, random_state=42)

5.2 Definir los parámetros de red

Los parámetros de red son:

**Tamaño del lote (Batch size):** define el número de muestras que se propagarán a través de la red. Mientras más grande el tamaño del lote, más memoria se ocupará.

**Número de clases(Num Classes):**  número total de clases posibles en la variable de destino

**Épocas (Epochs):** número total de iteraciones para las que se ejecutará el modelo cnn.

Por ejemplo, supongamos que tenemos 1050 muestras de entrenamiento y deseamos configurar un tamaño de lote igual a 100. 

El algoritmo toma las primeras 100 muestras (del 1 al 100) del conjunto de datos de entrenamiento y entrena la red. Luego, toma las segundas 100 muestras (del 101 al 200) y vuelve a entrenar la red. Podemos seguir haciendo este procedimiento hasta que hayamos propagado todas las muestras a través de la red. El problema podría ocurrir con el último conjunto de muestras. En nuestro ejemplo, hemos usado 1050 que no es divisible por 100 sin resto. La solución más simple es simplemente obtener las 50 muestras finales y entrenar la red.



**Ventajas de usar un tamaño de lote < número de todas las muestras:**

    Requiere menos memoria. Dado que entrena la red con menos muestras, el procedimiento de entrenamiento general requiere menos memoria. Eso es especialmente importante si no puede colocar todo el conjunto de datos en la memoria de su máquina.

    Por lo general, las redes se entrenan más rápido con mini lotes. Eso es porque actualizamos los pesos después de cada propagación. En nuestro ejemplo, propagamos 11 lotes (10 de ellos tenían 100 muestras y 1 tenía 50 muestras) y después de cada uno de ellos actualizamos los parámetros de nuestra red. Si usáramos todas las muestras durante la propagación, haríamos solo 1 actualización para el parámetro de la red.

Desventajas de usar un tamaño de lote < número de todas las muestras:

    Cuanto más pequeño sea el lote, menos precisa será la estimación del gradiente. En la figura a continuación, puede ver que la dirección del gradiente del mini lote (color verde) fluctúa mucho más en comparación con la dirección del gradiente del lote completo (color azul).
    

![img](https://i.stack.imgur.com/lU3sx.png)

In [57]:
# network parameters 
batch_size = 128
num_classes = 10
epochs = 5 # Further Fine Tuning can be done

# input image dimensions
img_rows, img_cols = 28, 28

5.3 Preprocesar las entradas

En el paso de preprocesamiento, los vectores de datos de imagen correspondientes se reforman en un vector de 4 dimensiones: tamaño total del lote, ancho de la imagen, altura de la imagen y canal. En nuestro caso, channel = 1 ya que solo usaremos un solo canal en lugar de tres canales (R, G, B). El siguiente paso es normalizar las entradas dividiéndolas por el valor máximo de píxeles, es decir 255.

In [65]:
# preprocess the train data 
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_train /= 255

# preprocess the validation data
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_val = x_val.astype('float32')
x_val /= 255

input_shape = (img_rows, img_cols, 1)

# convert the target variable 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

# preprocess the test data
Xtest = test.values
Xtest = Xtest.reshape(Xtest.shape[0], img_rows, img_cols, 1)

5.4 Crear la arquitectura del modelo CNN

En este paso, creamos la arquitectura de red neuronal convolucional con las siguientes capas:

    Capa convolucional con tamaño de núcleo = 3*3, 32 unidades convolucionales y función de activación RelU
    Capa convolucional con tamaño de kernel = 3*3, 64 unidades convolucionales y función de activación RelU
    Capa de agrupación (Poolong) máxima con tamaño de matriz de agrupación = 2*2
    Capa de abandono (Dropout): se utiliza una capa de abandono para regularizar y reducir el sobreajuste
    Capa aplanada (Flatten): una capa para convertir la salida en una matriz unidimensional
    Capa densa: una capa densa es una capa completamente conectada en la que cada nodo está conectado a todos los demás nodos en las capas anterior y siguiente. En nuestra red, contiene 128 neuronas, pero este número se puede cambiar para otros experimentos.
    Otra capa de abandono (dropout) para la regularización
    Capa de salida final: una capa densa con 10 neuronas para generar la clase de salida

En la red neuronal simple que implementamos en el paso 1, la función de pérdida era la función Log y el algoritmo de optimización era Gradient Descent. En esta red neuronal, usaremos **categorical_crossentropy**, ya que esta es una clasificación de varias clases, como función de pérdida y **Adadelta** como función de optimización.

In [66]:
model = Sequential()

# add first convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# add second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# add one max pooling layer 
model.add(MaxPooling2D(pool_size=(2, 2)))

# add one dropout layer
model.add(Dropout(0.25))

# add flatten layer
model.add(Flatten())

# add dense layer
model.add(Dense(128, activation='relu'))

# add another dropout layer
model.add(Dropout(0.5))

# add dense layer
model.add(Dense(num_classes, activation='softmax'))

# complile the model and view its architecur
model.compile(loss=keras.losses.categorical_crossentropy,  optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

5.5 Entrenamos

In [67]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val))
accuracy = model.evaluate(x_val, y_val, verbose=0)
print('Test accuracy:', accuracy[1])

Epoch 1/5
296/296 [==============================] - 27s 91ms/step - loss: 2.2962 - accuracy: 0.1233 - val_loss: 2.2679 - val_accuracy: 0.2529
Epoch 2/5
296/296 [==============================] - 30s 102ms/step - loss: 2.2552 - accuracy: 0.1963 - val_loss: 2.2215 - val_accuracy: 0.4152
Epoch 3/5
296/296 [==============================] - 30s 101ms/step - loss: 2.2089 - accuracy: 0.2828 - val_loss: 2.1684 - val_accuracy: 0.5086
Epoch 4/5
296/296 [==============================] - 32s 107ms/step - loss: 2.1549 - accuracy: 0.3533 - val_loss: 2.1038 - val_accuracy: 0.5788
Epoch 5/5
296/296 [==============================] - 33s 110ms/step - loss: 2.0887 - accuracy: 0.4167 - val_loss: 2.0242 - val_accuracy: 0.6255
Test accuracy: 0.6254761815071106


5.6 Generar Predicciones

In [69]:
pred = model.predict(Xtest)
y_classes = pred.argmax(axis=-1)
res = pd.DataFrame()
res['ImageId'] = list(range(1,28001))
res['Label'] = y_classes
res.to_csv("output.csv", index = False)

In [72]:
res

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,2
...,...,...
27995,27996,9
27996,27997,9
27997,27998,3
27998,27999,9
